Implementation of Deep Net with 1 hidden layer

In [1]:
pip install tenseal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 47.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import tenseal as ts
import pandas as pd
import random
from time import time

# those are optional and are not necessary for training
import numpy as np
import matplotlib.pyplot as plt

In [3]:
torch.random.manual_seed(73)
random.seed(73)


def split_train_test(x, y, test_ratio=0.3):
    idxs = [i for i in range(len(x))]
    random.shuffle(idxs)
    # delimiter between test and train data
    delim = int(len(x) * test_ratio)
    test_idxs, train_idxs = idxs[:delim], idxs[delim:]
    return x[train_idxs], y[train_idxs], x[test_idxs], y[test_idxs]


def heart_disease_data():
    data = pd.read_csv("/kaggle/input/trydata/framingham.csv")
    # drop rows with missing values
    data = data.dropna()
    # drop some features
    data = data.drop(columns=["education", "currentSmoker", "BPMeds", "diabetes", "diaBP", "BMI"])
    # balance data
    grouped = data.groupby('TenYearCHD')
    data = grouped.apply(lambda x: x.sample(grouped.size().min(), random_state=73).reset_index(drop=True))
    # extract labels
    y = torch.tensor(data["TenYearCHD"].values).float().unsqueeze(1)
    #data = data.drop("TenYearCHD",'columns')
    # standardize data
    data = (data - data.mean()) / data.std()
    x = torch.tensor(data.values).float()
    return split_train_test(x, y)


def random_data(m=1024, n=2):
    # data separable by the line `y = x`
    x_train = torch.randn(m, n)
    x_test = torch.randn(m // 2, n)
    y_train = (x_train[:, 0] >= x_train[:, 1]).float().unsqueeze(0).t()
    y_test = (x_test[:, 0] >= x_test[:, 1]).float().unsqueeze(0).t()
    return x_train, y_train, x_test, y_test


# You can use whatever data you want without modification to the tutorial
# x_train, y_train, x_test, y_test = random_data()
x_train, y_train, x_test, y_test = heart_disease_data()

print("############# Data summary #############")
print(f"x_train has shape: {x_train.shape}")
print(f"y_train has shape: {y_train.shape}")
print(f"x_test has shape: {x_test.shape}")
print(f"y_test has shape: {y_test.shape}")
print("#######################################")

############# Data summary #############
x_train has shape: torch.Size([780, 10])
y_train has shape: torch.Size([780, 1])
x_test has shape: torch.Size([334, 10])
y_test has shape: torch.Size([334, 1])
#######################################


<ipython-input-3-c85f04021ea0>:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = grouped.apply(lambda x: x.sample(grouped.size().min(), random_state=73).reset_index(drop=True))


In [4]:
# parameters
poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
# create TenSEALContext
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_training.global_scale = 2 ** 21
ctx_training.generate_galois_keys()

In [5]:
#t_start = time()
#enc_x_train = ts.ckks_tensor(ctx_training, x_train[0:300])
#enc_y_train = ts.ckks_tensor(ctx_training, y_train[0:300])
#t_end = time()
#print(f"Encryption of the training_set took {int(t_end - t_start)} seconds")

In [6]:
class EncryptedDL1layer() :
  def __init__(self) -> None:
      self.weight1=np.random.rand(10,3)*0.01
      self.bias1=np.random.rand(3)*0.01
      self.weight2=np.random.rand(3, 1)*0.01
      self.bias2=np.random.rand(1)*0.01
      self.dw1=0
      self.db1=0
      self.dw2=0
      self.db2=0
  def bootstrapping(enc,ctx_training) :
    return ts.ckks_tensor(ctx_training,enc.decrypt())
  def swiss(enc_x):
    return enc_x.polyval([0.6931,0.5,0.125,0])
  def swiss_derv(enc_x):
    return enc_x.polyval([0.5, 0.197, 0, -0.004])
  def forward(self,enc_x_train,ctx_training) :
    z11=enc_x_train.mm(self.weight1)
    z1=z11.add(self.bias1)
    a1=EncryptedDL1layer.swiss(z1)
    y=EncryptedDL1layer.bootstrapping(a1,ctx_training)
    z21=y.mm(self.weight2)
    z2=z21.add(self.bias2)
    a2=EncryptedDL1layer.swiss(z2)
    return a2,z2,a1,z1
  def backward(self,a2,z2,a1,z1,enc_y_train,ctx_training) :
    #calculating the output at the layer 2
    error=a2-enc_y_train
    der=EncryptedDL1layer.swiss_derv(z2)
    #finding delta2
    delta2=error.mul(der)
    #using bootstrapping
    delta2=EncryptedDL1layer.bootstrapping(delta2,ctx_training)
    #finding delta1
    del1=delta2.mm(self.weight2.transpose())
    del1=EncryptedDL1layer.bootstrapping(del1,ctx_training)
    #finding der1
    der1=EncryptedDL1layer.swiss_derv(z1)
    #using bootstrapping
    der1=EncryptedDL1layer.bootstrapping(der1,ctx_training)
    #finding delta1
    delta1=del1.mul(der1)
    #finding the gradients
    #for weight2 and bias2
    self.dw2=a1.transpose().mm(delta2)
    self.db2=delta2.sum()
    #for weight1 and bias1
    self.dw1=enc_x_train.transpose().mm(delta1)
    self.db1=delta1.sum()
  def update_params(self):
    self.weight2=self.weight2-0.01*self.dw2
    self.bias2=self.bias2-0.01*self.db2
    self.weight1=self.weight1-0.01*self.dw1
    self.bias1=self.bias1-0.01*self.db1
  def encrypt(self, context):
    self.weight1 = ts.ckks_tensor(context, self.weight1)
    self.bias1 = ts.ckks_tensor(context, self.bias1)
    self.weight2 = ts.ckks_tensor(context, self.weight2)
    self.bias2 = ts.ckks_tensor(context, self.bias2)
  def decrypt(self):
    self.weight1 = self.weight1.decrypt()
    self.bias1 = self.bias1.decrypt()
    self.weight2 = self.weight2.decrypt()
    self.bias2 = self.bias2.decrypt()
  def accuracy(self,x_test,y_test):
    #self.decrypt()
    w1 = torch.tensor(self.weight1)
    b1 = torch.tensor(self.bias1)
    out1 = torch.swiss(x_test.matmul(w1) + b1).reshape(-1, 1)
    w2 = torch.tensor(self.weight2)
    b2 = torch.tensor(self.bias2)
    out2 = torch.swiss(out1.matmul(w2) + b2).reshape(-1, 1)
    correct = torch.abs(y_test - out2) < 0.5
    return correct.float().mean()

In [7]:
EPOCHS = 5
eelr = EncryptedDL1layer()
times = []
for epoch in range(EPOCHS):
    print("Epoch : ",epoch)
    t_start = time()
    for i in range(0,780,60):
        print("Batch : ",i/60)
        t_start = time()
        enc_x_train = ts.ckks_tensor(ctx_training, x_train[i:i+60])
        enc_y_train = ts.ckks_tensor(ctx_training, y_train[i:i+60])
        t_end = time()
        print(f"Encryption of the training_batch took {int(t_end - t_start)} seconds")
        eelr.encrypt(ctx_training)
        a2,z2,a1,z1 = eelr.forward(enc_x_train,ctx_training)
        eelr.backward(a2,z2,a1,z1,enc_y_train,ctx_training)
        eelr.update_params()
        t_end = time()
        eelr.decrypt()
        print("Time taken for this batch is : ",t_end - t_start)
    times.append(t_end - t_start)
    a2,z2,a1,z1 = eelr.forward(enc_x_train,ctx_training)
    data=torch.tensor(a2.decrypt().tolist())
    loss_fn = torch.nn.BCEWithLogitsLoss()
    loss = loss_fn(data, y_train[i:i+60])
    print("Loss at epoch ",epoch,loss.data)
    out=torch.tensor(enc_y_train.sub(a2).decrypt().tolist())
    correct=torch.abs(out)<0.5
    print("Accuracy at epoch ",epoch,correct.float().mean())
print(f"\nAverage time per epoch: {int(sum(times) / len(times))} seconds")
print("Final weight1 ",eelr.weight1.tolist())
print("Final bias 1",eelr.bias1.tolist())
print("Final weight 2",eelr.weight2.tolist())
print("Final bias 2",eelr.bias2.tolist())

Epoch :  0
Batch :  0.0
Encryption of the training_batch took 4 seconds
Time taken for this batch is :  63.97819137573242
Batch :  1.0
Encryption of the training_batch took 3 seconds
Time taken for this batch is :  60.62110376358032
Batch :  2.0
Encryption of the training_batch took 3 seconds
Time taken for this batch is :  60.57720637321472
Batch :  3.0
Encryption of the training_batch took 3 seconds
Time taken for this batch is :  59.54231023788452
Batch :  4.0
Encryption of the training_batch took 3 seconds
Time taken for this batch is :  60.58647680282593
Batch :  5.0
Encryption of the training_batch took 3 seconds
Time taken for this batch is :  59.54318118095398
Batch :  6.0
Encryption of the training_batch took 3 seconds
Time taken for this batch is :  59.39877128601074
Batch :  7.0
Encryption of the training_batch took 3 seconds
Time taken for this batch is :  59.91908288002014
Batch :  8.0
Encryption of the training_batch took 3 seconds
Time taken for this batch is :  59.98209

In [8]:
#eelr.encrypt(ctx_training)
j,k,l,r=eelr.forward(enc_x_train,ctx_training)
#enc_y_train.sub(j).decrypt().tolist()

In [9]:
out=torch.tensor(enc_y_train.sub(j).decrypt().tolist())
correct=torch.abs(out)<0.5
print("Train Accuracy : ",correct.float().mean())

Train Accuracy :  tensor(0.9667)


In [10]:
enc_x_test = ts.ckks_tensor(ctx_training, x_test)
enc_y_test = ts.ckks_tensor(ctx_training, y_test)

In [11]:
m,n,j,k=eelr.forward(enc_x_test,ctx_training)
out=torch.tensor(enc_y_test.sub(m).decrypt().tolist())
correct=torch.abs(out)<0.5
print("Test Accuracy : ",correct.float().mean())

Test Accuracy :  tensor(0.9820)
